In [1]:
#4.1

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import numpy as np
from math import sqrt

In [3]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

In [4]:
cdatasub = pd.read_csv("london_flows.csv")

In [5]:
cdatasub.replace(0,0.0000001,inplace=True)

In [6]:
cdatasub.head(10)

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,1.000000e-07,599.0,78549.0,8131.525097
1,Abbey Road,Beckton,1.000000e+00,599.0,442.0,8510.121774
2,Abbey Road,Blackwall,3.000000e+00,599.0,665.0,3775.448872
3,Abbey Road,Canary Wharf,1.000000e+00,599.0,58772.0,5086.514220
4,Abbey Road,Canning Town,3.700000e+01,599.0,15428.0,2228.923167
5,Abbey Road,Crossharbour,1.000000e+00,599.0,1208.0,6686.475560
6,Abbey Road,Custom House,1.000000e-07,599.0,845.0,3824.855630
7,Abbey Road,Cutty Sark,2.000000e+00,599.0,1748.0,8503.898909
8,Abbey Road,Cyprus,7.000000e+00,599.0,850.0,6532.099618
9,Abbey Road,Devons Road,1.000000e+00,599.0,611.0,3958.324171


In [7]:
cdatasubmat = pd.pivot_table(cdatasub, values ="flows", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.000000,5.990000e+02
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,NaN,NaN,1.224000e+03
Acton Town,NaN,NaN,NaN,3.000000e+00,1.700000e+01,NaN,3.500000e+01,1.000000e-07,NaN,11.000000,...,7.700000e+01,3.000000e+00,6.000000e+00,9.000000,NaN,1.000000e-07,NaN,1.000000e-07,NaN,3.745000e+03
Aldgate,NaN,NaN,1.000000e-07,NaN,1.000000e-07,NaN,NaN,1.000000e-07,NaN,17.000000,...,1.000000e-07,NaN,4.000000e+00,8.000000,NaN,1.000000e-07,NaN,1.000000e-07,NaN,2.886000e+03
Aldgate East,NaN,NaN,2.000000e+00,1.000000e-07,NaN,NaN,1.000000e-07,1.000000e-07,NaN,20.000000,...,2.400000e+01,1.000000e-07,1.000000e-07,12.000000,NaN,1.000000e+00,NaN,1.000000e+00,NaN,3.172000e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.000000e+00,5.000000e+00,4.700000e+01,NaN,NaN,NaN,NaN,22.000000,...,2.000000e+00,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,4.868000e+03
Woodgrange Park,NaN,1.000000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.300000e+02
Woodside Park,NaN,NaN,1.000000e+00,2.600000e+01,1.100000e+01,NaN,1.000000e-07,NaN,NaN,59.000000,...,1.000000e-07,NaN,1.000000e-07,NaN,NaN,NaN,NaN,NaN,NaN,3.093000e+03


In [8]:


import statsmodels.api as sm
import statsmodels.formula.api as smf
x_variables = ["population", "jobs", "distance"]
log_x_vars = []
for x in x_variables:
    cdatasub[f"log_{x}"] = np.log(cdatasub[x])
    log_x_vars.append(f"log_{x}")

#create the formula (the "-1" indicates no intercept in the regression model).
formula = 'flows ~ station_origin + log_jobs + log_distance-1'
#run a production constrained sim
prodSim = smf.glm(formula = formula, data=cdatasub, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61073
Model Family:                 Poisson   Df Model:                          400
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.2592e+06
Date:                Mon, 02 May 2022   Deviance:                   2.3462e+06
Time:                        14:22:11   Pearson chi2:                 4.76e+06
No. Iterations:                    22                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [9]:
#create some Oi and Dj columns in the dataframe and store row and column totals in them:
#to create O_i, take cdatasub ...then... group by origcodenew ...then... summarise by calculating the sum of Total
O_i = pd.DataFrame(cdatasub.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
cdatasub = cdatasub.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(cdatasub.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
cdatasub = cdatasub.merge(D_j, on = "station_destination", how = "left" )

In [10]:
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(prodSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "")
#then once you have done this you can join them back into the dataframes
cdatasub = cdatasub.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
cdatasub.drop(columns = ["coef"], inplace = True)
#check this has worked
cdatasub.head()

/tmp/ipykernel_450/3738776740.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  coefs["coef"] = coefs["coef"].str.replace(x, "")


,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i
0,Abbey Road,Bank and Monument,1.000000e-07,599.0,78549.0,8131.525097,6.395262,11.271478,9.003504,599.0,78549.000001,-3.823705
1,Abbey Road,Beckton,1.000000e+00,599.0,442.0,8510.121774,6.395262,6.091310,9.049012,599.0,442.000001,-3.823705
2,Abbey Road,Blackwall,3.000000e+00,599.0,665.0,3775.448872,6.395262,6.499787,8.236275,599.0,665.000001,-3.823705
3,Abbey Road,Canary Wharf,1.000000e+00,599.0,58772.0,5086.514220,6.395262,10.981421,8.534348,599.0,58772.000001,-3.823705
4,Abbey Road,Canning Town,3.700000e+01,599.0,15428.0,2228.923167,6.395262,9.643939,7.709274,599.0,15428.000004,-3.823705


In [11]:
alpha_i = prodSim.params[0:399]
gamma = prodSim.params[399]
beta = -prodSim.params[400]

In [12]:
beta

0.058657419649845735

In [13]:
gamma

0.8273854119977159

In [14]:
alpha_i.head(10)

station_origin[Abbey Road]      -3.823705
station_origin[Acton Central]   -3.121902
station_origin[Acton Town]      -3.702973
station_origin[Aldgate]         -4.028844
station_origin[Aldgate East]    -3.948172
station_origin[All Saints]      -3.673999
station_origin[Alperton]        -4.378019
station_origin[Amersham]        -4.667449
station_origin[Anerley]         -2.986040
station_origin[Angel]           -3.664923
dtype: float64

In [15]:
cdatasub["prodsimest1"] = np.exp(cdatasub["alpha_i"]+gamma*cdatasub["log_jobs"] 
                                 - beta*cdatasub["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
cdatasub.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i,prodsimest1
0,Abbey Road,Bank and Monument,1.000000e-07,599.0,78549.0,8131.525097,6.395262,11.271478,9.003504,599.0,78549.000001,-3.823705,144.608400
1,Abbey Road,Beckton,1.000000e+00,599.0,442.0,8510.121774,6.395262,6.091310,9.049012,599.0,442.000001,-3.823705,1.984495
2,Abbey Road,Blackwall,3.000000e+00,599.0,665.0,3775.448872,6.395262,6.499787,8.236275,599.0,665.000001,-3.823705,2.918313
3,Abbey Road,Canary Wharf,1.000000e+00,599.0,58772.0,5086.514220,6.395262,10.981421,8.534348,599.0,58772.000001,-3.823705,116.928176
4,Abbey Road,Canning Town,3.700000e+01,599.0,15428.0,2228.923167,6.395262,9.643939,7.709274,599.0,15428.000004,-3.823705,40.582782
5,Abbey Road,Crossharbour,1.000000e+00,599.0,1208.0,6686.475560,6.395262,7.096721,8.807842,599.0,1208.000000,-3.823705,4.624525
6,Abbey Road,Custom House,1.000000e-07,599.0,845.0,3824.855630,6.395262,6.739337,8.249276,599.0,845.000002,-3.823705,3.555312
7,Abbey Road,Cutty Sark,2.000000e+00,599.0,1748.0,8503.898909,6.395262,7.466228,9.048280,599.0,1748.000000,-3.823705,6.190367
8,Abbey Road,Cyprus,7.000000e+00,599.0,850.0,6532.099618,6.395262,6.745236,8.784484,599.0,850.000001,-3.823705,3.462290
9,Abbey Road,Devons Road,1.000000e+00,599.0,611.0,3958.324171,6.395262,6.415097,8.283576,599.0,611.000001,-3.823705,2.713284


In [16]:
#first round the estimates
cdatasub["prodsimest1"] = round(cdatasub["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
cdatasubmat3 = cdatasub.pivot_table(values ="prodsimest1", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat3

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,601.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1227.0
Acton Town,NaN,NaN,NaN,23.0,23.0,NaN,4.0,1.0,NaN,24.0,...,19.0,3.0,8.0,7.0,NaN,3.0,NaN,3.0,NaN,3741.0
Aldgate,NaN,NaN,6.0,NaN,20.0,NaN,NaN,1.0,NaN,19.0,...,14.0,NaN,6.0,5.0,NaN,2.0,NaN,2.0,NaN,2888.0
Aldgate East,NaN,NaN,6.0,21.0,NaN,NaN,3.0,1.0,NaN,20.0,...,15.0,2.0,6.0,5.0,NaN,3.0,NaN,3.0,NaN,3176.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,11.0,31.0,32.0,NaN,NaN,NaN,NaN,32.0,...,25.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,4870.0
Woodgrange Park,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529.0
Woodside Park,NaN,NaN,7.0,20.0,21.0,NaN,3.0,NaN,NaN,21.0,...,17.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,3099.0


In [28]:
cdatasubmat

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.000000,5.990000e+02
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,NaN,NaN,1.224000e+03
Acton Town,NaN,NaN,NaN,3.000000e+00,1.700000e+01,NaN,3.500000e+01,1.000000e-07,NaN,11.000000,...,7.700000e+01,3.000000e+00,6.000000e+00,9.000000,NaN,1.000000e-07,NaN,1.000000e-07,NaN,3.745000e+03
Aldgate,NaN,NaN,1.000000e-07,NaN,1.000000e-07,NaN,NaN,1.000000e-07,NaN,17.000000,...,1.000000e-07,NaN,4.000000e+00,8.000000,NaN,1.000000e-07,NaN,1.000000e-07,NaN,2.886000e+03
Aldgate East,NaN,NaN,2.000000e+00,1.000000e-07,NaN,NaN,1.000000e-07,1.000000e-07,NaN,20.000000,...,2.400000e+01,1.000000e-07,1.000000e-07,12.000000,NaN,1.000000e+00,NaN,1.000000e+00,NaN,3.172000e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.000000e+00,5.000000e+00,4.700000e+01,NaN,NaN,NaN,NaN,22.000000,...,2.000000e+00,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,4.868000e+03
Woodgrange Park,NaN,1.000000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.300000e+02
Woodside Park,NaN,NaN,1.000000e+00,2.600000e+01,1.100000e+01,NaN,1.000000e-07,NaN,NaN,59.000000,...,1.000000e-07,NaN,1.000000e-07,NaN,NaN,NaN,NaN,NaN,NaN,3.093000e+03


In [34]:
eval('1.542391e+06')

1542391.0

In [18]:
#加入条件，变Jobs

cdatasub["jobsnew"] = cdatasub.loc[cdatasub["station_origin"] == "Canary Wharf", "jobs"] *0.5


In [19]:
cdatasub[cdatasub["station_origin"] == "Canary Wharf"].head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i,prodsimest1,jobsnew
8805,Canary Wharf,Abbey Road,2.000000e+00,14632.0,345.0,5086.514220,9.590966,5.843544,8.534348,14632.000006,345.000002,-2.547943,6.0,172.5
8806,Canary Wharf,Acton Town,7.000000e+00,14632.0,2202.0,20398.165882,9.590966,7.697121,9.923200,14632.000006,2202.000004,-2.547943,25.0,1101.0
8807,Canary Wharf,Aldgate,1.000000e-07,14632.0,7782.0,6564.419680,9.590966,8.959569,8.789419,14632.000006,7782.000002,-2.547943,77.0,3891.0
8808,Canary Wharf,Aldgate East,3.000000e+00,14632.0,7932.0,5127.998899,9.590966,8.978660,8.542471,14632.000006,7932.000002,-2.547943,80.0,3966.0
8809,Canary Wharf,All Saints,2.700000e+01,14632.0,444.0,1340.088733,9.590966,6.095825,7.200491,14632.000006,444.000001,-2.547943,8.0,222.0
8810,Canary Wharf,Alperton,1.000000e+00,14632.0,741.0,25371.253826,9.590966,6.608001,10.141372,14632.000006,741.000008,-2.547943,10.0,370.5
8811,Canary Wharf,Amersham,1.000000e-07,14632.0,256.0,49907.635676,9.590966,5.545177,10.817929,14632.000006,256.000012,-2.547943,4.0,128.0
8812,Canary Wharf,Angel,2.400000e+01,14632.0,8103.0,8538.426368,9.590966,8.999990,9.052332,14632.000006,8103.000001,-2.547943,79.0,4051.5
8813,Canary Wharf,Archway,5.000000e+00,14632.0,2193.0,14463.519883,9.590966,7.693026,9.579385,14632.000006,2193.000006,-2.547943,26.0,1096.5
8814,Canary Wharf,Arnos Grove,1.000000e+00,14632.0,748.0,19708.927000,9.590966,6.617403,9.888827,14632.000006,748.000011,-2.547943,10.0,374.0


In [20]:
cdatasub["prodsimest2"] = np.exp(cdatasub["alpha_i"]+gamma*np.log(cdatasub["jobsnew"]) - beta*cdatasub["log_distance"])

cdatasub["prodsimest2"] = round(cdatasub["prodsimest2"],0)
#now we can convert the pivot table into a matrix
cdatasubmat4 = cdatasub.pivot_table(values ="prodsimest2", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat4

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
Acton Town,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN
Aldgate,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN
Aldgate East,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Woodgrange Park,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Woodside Park,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:

#calculate some new wj^alpha and d_ij^beta values
Dj2_gamma = cdatasub["jobs"]**gamma
dist_beta = cdatasub["distance"]**beta
#calcualte the first stage of the Ai values
cdatasub["Ai1"] = Dj2_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(cdatasub.groupby(["station_origin"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i"}, inplace=True)
#and write the A_i values back into the dataframe
cdatasub = cdatasub.merge(A_i, left_on="station_origin", right_index=True, how="left")

In [22]:
#to check everything works, recreate the original estimates
cdatasub["prodsimest3"] = cdatasub["A_i"]*cdatasub["O_i"]*Dj2_gamma*dist_beta
#round
cdatasub["prodsimest3"] = round(cdatasub["prodsimest3"])
#check
cdatasub[["prodsimest1", "prodsimest3"]]

,prodsimest1,prodsimest3
0,145.0,158.0
1,2.0,2.0
2,3.0,3.0
3,117.0,121.0
4,41.0,38.0
...,...,...
61469,139.0,143.0
61470,216.0,211.0
61471,25.0,24.0
61472,49.0,46.0


In [23]:
#calculate some new wj^alpha and d_ij^beta values
Dj3_gamma = cdatasub["jobsnew"]**gamma
#calcualte the first stage of the Ai values
cdatasub["Ai1"] = Dj3_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(cdatasub.groupby(["station_origin"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i2"}, inplace=True)
#and write the A_i values back into the dataframe
cdatasub = cdatasub.merge(A_i, left_on="station_origin", right_index=True, how="left")

In [24]:
#to check everything works, recreate the original estimates
cdatasub["prodsimest4"] = cdatasub["A_i2"]*cdatasub["O_i"]*Dj3_gamma*dist_beta
#round
cdatasub["prodsimest4"] = round(cdatasub["prodsimest4"])

In [37]:
cdatasubmat5 = cdatasub.pivot_table(values ="prodsimest4", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat5

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
Acton Town,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN
Aldgate,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN
Aldgate East,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Woodgrange Park,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Woodside Park,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
cdatasubmat5["Canary Wharf"]

station_origin
Abbey Road            0.0
Acton Central         NaN
Acton Town            0.0
Aldgate               0.0
Aldgate East          0.0
                    ...  
Woodford              0.0
Woodgrange Park       NaN
Woodside Park         0.0
Woolwich Arsenal      0.0
All                 104.0
Name: Canary Wharf, Length: 400, dtype: float64

In [50]:
cdatasubmat["Canary Wharf"]

station_origin
Abbey Road              1.000000
Acton Central                NaN
Acton Town             57.000000
Aldgate                 1.000000
Aldgate East            3.000000
                        ...     
Woodford              192.000000
Woodgrange Park              NaN
Woodside Park          42.000000
Woolwich Arsenal       82.000000
All                 58772.000001
Name: Canary Wharf, Length: 400, dtype: float64